In [1]:
!git add 'production_pipeline.ipynb'
!git commit -m 'add processing notebook'
!git push

[main dae5fc6] add processing notebook
 1 file changed, 6 insertions(+)
 create mode 100644 production_pipeline.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 355 bytes | 355.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/gaylejuntilla/MA3831-A3.git
   af0b5f6..dae5fc6  main -> main


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [18]:
# reading datasets
guardian = pd.read_csv('guardian_articles.csv')
abc = pd.read_csv('abc_news_articles.csv')
skynews = pd.read_csv('skynews_articles_full.csv')

In [61]:
def clean_raw_data(df, source):
    df = df.copy()
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    string_columns = ['headline', 'title', 'content']
    for column in string_columns:
        df[column] = df[column].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())
        df[column] = df[column].apply(lambda x: re.sub(r'\d+', '', str(x)))
    df['date'] = df['date'].apply(parse_date)
    df['date'] = df['date'].dt.date
    df.reset_index(drop=True, inplace=True)
    df['source'] = source
    return df

In [62]:
def parse_date(date_str):
    try:
        # Try parsing as date-time format (like '2025-04-01 18:18:43')
        return pd.to_datetime(date_str, errors='raise')  # Raise error if parsing fails
    except Exception:
        try:
            # If the first attempt fails, try date-only format (like '10 March 2025')
            return pd.to_datetime(date_str, format='%d %B %Y', errors='coerce')
        except Exception:
            # If both attempts fail, return NaT (Not a Time)
            return pd.NaT

In [63]:
guardian_df = clean_raw_data(guardian, 'theguardian')
abc_df = clean_raw_data(abc, 'abc')
skynews_df = clean_raw_data(skynews, 'skynews')

In [64]:
guardian_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  176 non-null    object
 1   title     176 non-null    object
 2   content   176 non-null    object
 3   date      176 non-null    object
 4   url       176 non-null    object
 5   source    176 non-null    object
dtypes: object(6)
memory usage: 8.4+ KB


In [65]:
abc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  160 non-null    object
 1   title     160 non-null    object
 2   date      160 non-null    object
 3   content   160 non-null    object
 4   url       160 non-null    object
 5   source    160 non-null    object
dtypes: object(6)
memory usage: 7.6+ KB


In [66]:
skynews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  850 non-null    object
 1   title     850 non-null    object
 2   content   850 non-null    object
 3   date      850 non-null    object
 4   url       850 non-null    object
 5   source    850 non-null    object
dtypes: object(6)
memory usage: 40.0+ KB


In [67]:
new_column_order = ['headline', 'title', 'content', 'date', 'url', 'source']

# Reorder the columns
abc_df = abc_df[new_column_order]
abc_df

,headline,title,content,date,url,source
0,How a 'chopper on a stick' entered the federal...,Deputy PM promises return of Nowra's chopper o...,An Iroquois helicopter that stood proudly on ...,2025-04-01,https://www.abc.net.au/news/2025-04-01/nowra-c...,abc
1,Teal candidate banned from hair salon over 'po...,Teal candidate banned from Sydney salon after ...,Teal independent candidate Nicolette Boele has...,2025-04-01,https://www.abc.net.au/news/2025-04-01/teal-in...,abc
2,Posters of politicians and billionaires as Naz...,"McCormack slams posters of himself, Dutton, Ri...",Former Deputy Prime Minister Michael McCormack...,2025-04-01,https://www.abc.net.au/news/2025-04-01/former-...,abc
3,"Albanese won't budge on key US grievances, as ...","Albanese won't budge on key US grievances, as ...",An updated list of grievances published by the...,2025-04-01,https://www.abc.net.au/news/2025-04-01/electio...,abc
4,Coalition promises to relax home lending rules...,Coalition promises to relax home lending rules...,The Coalition is promising to relax rules arou...,2025-04-01,https://www.abc.net.au/news/2025-04-01/electio...,abc
...,...,...,...,...,...,...
155,Community desperate for investment welcomes $m...,'Forgotten' Wyndham welcomes federal pledge to...,Community leaders in Western Australia's north...,2025-01-11,https://www.abc.net.au/news/2025-01-11/albanes...,abc
156,A week of signals for the wild way politics mi...,Albanese's Bruce Highway announcement got as a...,If you were constructing your federal election...,2025-01-11,https://www.abc.net.au/news/2025-01-11/albanes...,abc
157,Albanese looks revived in the heat as he reset...,From 'no Australian left behind' to 'no Austra...,It's north of degrees Celsius. The humidity i...,2025-01-10,https://www.abc.net.au/news/2025-01-10/albanes...,abc
158,Albanese enters this election 'kicking against...,Albanese is running at the election 'kicking a...,Anthony Albanese could have spent Christmas at...,2025-01-10,https://www.abc.net.au/news/2025-01-10/albanes...,abc


In [68]:
# selecting every 5th row to reduce size
skynews_df = skynews_df.iloc[::5]

In [69]:
skynews_df

,headline,title,content,date,url,source
0,WATCH LIVE: Shadow treasurer Angus Taylor addr...,WATCH LIVE: Shadow treasurer Angus Taylor addr...,Shadow treasurer Angus Taylor is delivering hi...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews
5,New polling reveals Jacinta Allan’s support pl...,Polls reveals sharp drop in support for Jacint...,New polling has revealed support for Jacinta A...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews
10,Albanese’s major election push for group of Au...,Federal Election : Labor to make submission to...,Labor will push the Fair Work Commission to gi...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews
15,Coalition MP’s corflutes defaced with abhorren...,'This is not Australian': Federal Election tak...,"Andrew Wallace, the federal LNP member for Fis...",2025-04-01,https://www.skynews.com.au/australia-news/poli...,skynews
20,Fresh polling spells trouble for Allegra Spend...,Fresh polling spells trouble for Teal MP Alleg...,Fresh polling spells trouble for Teal MP Alleg...,2025-04-01,https://www.skynews.com.au/australia-news/poli...,skynews
...,...,...,...,...,...,...
825,‘Lost control’: Albanese govt overlooks $bn ta...,Albanese government faces $. billion budget bl...,The Albanese government has left a gaping $. b...,2025-01-08,https://www.skynews.com.au/australia-news/poli...,skynews
830,‘Persona non grata’: Huge call on Rudd as Trum...,Australia's Ambassador to the US Kevin Rudd fa...,As leaders around the globe have prepared for ...,2025-01-07,https://www.skynews.com.au/australia-news/poli...,skynews
835,‘This is their job’: Qld mayor calls out Alban...,Mackay Mayor Greg Williamson says Bruce Highwa...,The Albanese government has been criticised fo...,2025-01-07,https://www.skynews.com.au/australia-news/poli...,skynews
840,Labor plunges $.bn to upgrade one of Australia...,Albanese government gives $. billion funding t...,Prime Minister Anthony Albanese has used his f...,2025-01-06,https://www.skynews.com.au/australia-news/poli...,skynews


In [70]:
guardian_df

,headline,title,content,date,url,source
0,RBA holds rates at .% but Michele Bullock says...,RBA holds rates at .% but Michele Bullock says...,Reserve Bank of Australia governor says board ...,2025-04-01,https://www.theguardian.com/australia-news/202...,theguardian
1,Gas producers question whether Coalition’s ene...,Gas producers question whether Coalition’s ene...,Industry leaders say there is little detail on...,2025-04-01,https://www.theguardian.com/australia-news/202...,theguardian
2,What do Peter Dutton’s proposed funding cuts r...,What do Peter Dutton’s proposed funding cuts r...,Here’s what you need to know about the opposit...,2025-04-01,https://www.theguardian.com/australia-news/202...,theguardian
3,Labor accuses Dutton of copying Trump with sug...,Labor accuses Dutton of copying Trump with sug...,Education minister criticises ‘extreme’ Coalit...,2025-04-01,https://www.theguardian.com/australia-news/202...,theguardian
4,Are our politicians on the right track?,Are our politicians on the right track?,It sure looks like familiar territory,2025-04-01,https://www.theguardian.com/commentisfree/pict...,theguardian
...,...,...,...,...,...,...
171,Albanese says Coalition ‘more conservative tha...,Albanese says Coalition ‘more conservative tha...,Prime minister and opposition leader start ele...,2025-01-13,https://www.theguardian.com/australia-news/202...,theguardian
172,The Australian election is coming: what moment...,The Australian election is coming: what moment...,There is still no date for the federal electio...,2025-01-13,https://www.theguardian.com/australia-news/aud...,theguardian
173,Liberal faction fears Warren Mundine will sink...,Liberal faction fears Warren Mundine will sink...,Anti-voice campaigner will face off with Gisel...,2025-01-11,https://www.theguardian.com/australia-news/202...,theguardian
174,Anthony Albanese switches to election footing ...,Anthony Albanese switches to election footing ...,PM frames federal poll as choice between ‘buil...,2025-01-06,https://www.theguardian.com/australia-news/202...,theguardian


In [71]:
corpus = pd.concat([guardian_df, abc_df, skynews_df], ignore_index=True)
corpus = corpus.sort_values(by='date', ascending=False)
corpus

,headline,title,content,date,url,source
337,New polling reveals Jacinta Allan’s support pl...,Polls reveals sharp drop in support for Jacint...,New polling has revealed support for Jacinta A...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews
336,WATCH LIVE: Shadow treasurer Angus Taylor addr...,WATCH LIVE: Shadow treasurer Angus Taylor addr...,Shadow treasurer Angus Taylor is delivering hi...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews
338,Albanese’s major election push for group of Au...,Federal Election : Labor to make submission to...,Labor will push the Fair Work Commission to gi...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews
341,Albanese hits back at Trump’s list of grievanc...,Anthony Albanese pressed on relationship with ...,Prime Minister Anthony Albanese has insisted h...,2025-04-01,https://www.skynews.com.au/australia-news/poli...,skynews
181,Bipartisan support to invest $m for health tra...,Coalition to match Labor's $ million pledge to...,A $ million investment into a new healthcare c...,2025-04-01,https://www.abc.net.au/news/2025-03-31/federal...,abc
...,...,...,...,...,...,...
335,Albanese unveils $.b highway upgrade as electi...,Federal Labor pledges to fund bulk of Bruce Hi...,The federal government has announced it will f...,2025-01-06,https://www.abc.net.au/news/2025-01-06/federal...,abc
504,Labor plunges $.bn to upgrade one of Australia...,Albanese government gives $. billion funding t...,Prime Minister Anthony Albanese has used his f...,2025-01-06,https://www.skynews.com.au/australia-news/poli...,skynews
174,Anthony Albanese switches to election footing ...,Anthony Albanese switches to election footing ...,PM frames federal poll as choice between ‘buil...,2025-01-06,https://www.theguardian.com/australia-news/202...,theguardian
505,‘A huge hit for any family’: New emissions sta...,‘A huge hit for any family’: Coalition warns n...,The federal Coalition is warning Australian co...,2025-01-02,https://www.skynews.com.au/australia-news/poli...,skynews


In [ ]:
corpus.reset_index(drop=True, inplace=)